In [1]:
%matplotlib inline
import os
import glob
import pprint
from error_analysis import *
import matplotlib.pyplot as plt

In [2]:
def latest(model_dir):
    return sorted(glob.glob(os.path.join(model_dir, '*')))[-1]
    
latest('../export/dnn-full')

def example_gen():
    with open('../data/votes-mini.dat') as fp:
        for line in fp:
            yield eval(line)

analysis = ErrorAnalysis({'pred': pred_error}, example_generator_fn=example_gen, batch_size=10)
errors = analysis.analyze(latest('../export/dnn-full'), 'dnn-full.errors')

INFO:tensorflow:Restoring parameters from ../export/dnn-full/1504490986/variables/variables
Default SIGNATURE_DEF:
inputs {
  key: "BillTitle"
  value {
    name: "Placeholder:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
inputs {
  key: "SponsorParty"
  value {
    name: "input/batch_1:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
inputs {
  key: "VoterAge"
  value {
    name: "input/batch_3:0"
    dtype: DT_INT64
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
inputs {
  key: "VoterParty"
  value {
    name: "input/batch_2:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
inputs {
  key: "VoterState"
  value {
    name: "input/batch:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
outputs {
  key: "aye"
  value {
    name: "GreaterEqual:0"
    dtype: DT_BOOL
    tensor_shape {
      dim {
 

In [3]:
print errors[0]['example'].keys()
count = 0
def different_party(e):
    return e['example']['VoterParty'] != e['example']['SponsorParty']

filt_errors = filter(different_party, errors)

['BillTitle', 'VoterState', 'SponsorParty', 'VoterParty', 'VoterAge', 'Decision']


In [4]:
from IPython.display import HTML, display

def html_table(errors):
    if not errors:
        return ""

    keys = errors[0]['example'].keys()
    s = "<table>"
    s += "<tr><th>" + "</th><th>".join(keys) + "</th></tr>"
    row_template = "<tr><td>"+"</td><td>".join("{"+key+"}" for key in keys) + "</td></tr>"
    s += "".join(row_template.format(**error['example']) for error in errors)
    s += "</table>"
    return s

def show_table(errors):
    display(HTML(html_table(errors)))
    
show_table(filt_errors)

BillTitle,VoterState,SponsorParty,VoterParty,VoterAge,Decision
"Making appropriations for military construction, family housing, and base realignment and closure for the Department of Defense for the fiscal year ending September 30, 1997, and for other purposes.",LA,republican,democrat,73,Aye
"Making appropriations for military construction, family housing, and base realignment and closure for the Department of Defense for the fiscal year ending September 30, 1997, and for other purposes.",SC,republican,democrat,95,Aye
"Making appropriations for the Departments of Labor, Health and Human Services, and Education, and related agencies, for the fiscal year ending September 30, 1997, and for other purposes.",CA,republican,democrat,69,Aye
A bill to grant the power to the President to reduce budget authority.,RI,republican,democrat,98,Nay
"Making appropriations for military construction, family housing, and base realignment and closure for the Department of Defense for the fiscal year ending September 30, 1997, and for other purposes.",OR,republican,democrat,68,Aye
A bill to grant the power to the President to reduce budget authority.,NE,republican,democrat,74,Nay
"Making appropriations for military construction, family housing, and base realignment and closure for the Department of Defense for the fiscal year ending September 30, 1997, and for other purposes.",IL,republican,democrat,88,Aye
"Making appropriations for military construction, family housing, and base realignment and closure for the Department of Defense for the fiscal year ending September 30, 1997, and for other purposes.",WI,republican,democrat,64,Aye
A bill to grant the power to the President to reduce budget authority.,NV,republican,democrat,77,Nay
A bill to grant the power to the President to reduce budget authority.,MI,republican,democrat,83,Nay


In [5]:
# Need a TFRecord example generator.
import tensorflow as tf


def read_graph(filename):
    reader = tf.TFRecordReader()
    filename_queue = tf.train.string_input_producer(tf.constant([filename]))
    _, record = reader.read(tf.train.string_input_producer(tf.constant([filename])))
    feature_spec = {}
    feature_spec['BillTitle'] = tf.FixedLenSequenceFeature([], tf.int64, allow_missing=True)
    feature_spec['VoterState'] = tf.FixedLenFeature([], tf.string)
    feature_spec['VoterParty'] = tf.FixedLenFeature([], tf.string)
    feature_spec['SponsorParty'] = tf.FixedLenFeature([], tf.string)
    feature_spec['VoterAge'] = tf.FixedLenFeature([], tf.int64)
    feature_spec['Decision'] = tf.FixedLenFeature([], tf.string)
    example = tf.parse_single_example(record, feature_spec)
    return example

def tfrecord_generator():
    example = read_graph("../mini-data/valid.tfrecord")
    try:
        with tf.Session() as sess:
            tf.train.start_queue_runners(sess=sess)
            sess.run(tf.global_variables_initializer())
            while True:
                x = sess.run(example)
                # Inference time takes a string! This is the *wrong* string though.
                x['BillTitle'] = str(x['BillTitle'])
                yield x
    except:
        return

# Alternatively we can try to work directly with the proto.
# record_iter = tf.python_io.tf_record_iterator("../mini-data/valid.tfrecord")

analysis = ErrorAnalysis({'pred': pred_error}, example_generator_fn=tfrecord_generator, batch_size=10)
errors = analysis.analyze(latest('../export/dnn-full'), 'dnn-full.errors')

INFO:tensorflow:Restoring parameters from ../export/dnn-full/1504490986/variables/variables
Default SIGNATURE_DEF:
inputs {
  key: "BillTitle"
  value {
    name: "Placeholder:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
inputs {
  key: "SponsorParty"
  value {
    name: "input/batch_1:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
inputs {
  key: "VoterAge"
  value {
    name: "input/batch_3:0"
    dtype: DT_INT64
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
inputs {
  key: "VoterParty"
  value {
    name: "input/batch_2:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
inputs {
  key: "VoterState"
  value {
    name: "input/batch:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
outputs {
  key: "aye"
  value {
    name: "GreaterEqual:0"
    dtype: DT_BOOL
    tensor_shape {
      dim {
 

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/training/queue_runner_impl.py", line 238, in _run
    enqueue_callable()
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 1063, in _single_operation_run
    target_list_as_strings, status, None)
  File "/usr/lib/python2.7/contextlib.py", line 24, in __exit__
    self.gen.next()
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/framework/errors_impl.py", line 466, in raise_exception_on_not_ok_status
    pywrap_tensorflow.TF_GetCode(status))
CancelledError: Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job

KeyboardInterrupt: 